In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

# Recover Collection Strategy

## Calculations

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Descargas/contactability_scores_(4)_(2).csv")

In [24]:
df

,customer_id,contactability_score,total_debt,interest,Cost_of_call,debt_plus_interest,probability_of_payment,recover_probability,recover_account
0,1,0.293,1000,150,30,1150,0.75,0.21975,0
1,2,0.250,1000,150,30,1150,0.75,0.18750,0
2,3,0.033,1000,150,30,1150,0.75,0.02475,0
3,4,0.239,1000,150,30,1150,0.75,0.17925,0
4,5,0.681,1000,150,30,1150,0.75,0.51075,1
...,...,...,...,...,...,...,...,...,...
670,671,0.463,1000,150,30,1150,0.75,0.34725,1
671,672,0.434,1000,150,30,1150,0.75,0.32550,1
672,673,0.396,1000,150,30,1150,0.75,0.29700,1
673,674,0.230,1000,150,30,1150,0.75,0.17250,0


In [3]:
df.shape

(675, 4)

In [4]:
df['Cost_of_call'] = 30
df['debt_plus_interest'] = df.total_debt + df.interest
df['probability_of_payment'] = 0.75
df['recover_probability'] = df.contactability_score*df.probability_of_payment

## Visualization

In [27]:
fig = px.box(df.recover_probability, points='all')
fig.update_xaxes(title="Recover Probability")

In [6]:
threshold = 0.28 # Q3

df['recover_account'] = df.recover_probability >= threshold
df['recover_account'] = df.recover_account.astype('int')

In [14]:
recovered_accounts = df[df.recover_account == 1]
lost_accounts = df[df.recover_account == 0]

## Answers

### Recovered accounts (accounts we get out of delinquency).

In [12]:
total_recovered_accounts = df.recover_account.sum()
print("Recovered accounts: ", total_recovered_accounts ,"accounts")

Recovered accounts:  341 accounts


### Recovered total debt.

In [15]:
recovered_total_debt = recovered_accounts['total_debt'].sum()
print("Recovered total debt: $", recovered_total_debt)

Recovered total debt: $ 341000


### Revenue coming from collected interest. (Note that the total debt collected is not included in revenue, why?).


In [17]:
revenue_coming_collected_interest = recovered_accounts['interest'].sum()
print("Revenue coming from collected interest: $", revenue_coming_collected_interest)

Revenue coming from collected interest: $ 51150


### Total campaign cost.

In [18]:
campaign_cost = df.Cost_of_call.sum()
print("Total Campaign Cost: $", campaign_cost)

Total Campaign Cost: $ 20250


### DQ losses coming from uncollected total debt.

In [21]:
lost_accounts['total_debt'].sum()

334000

### Collections income.

In [22]:
recovered_accounts['debt_plus_interest'].sum() - campaign_cost

371900

In [44]:
df['Strategy Result'] = df['recover_account'].map(lambda x: 'Recovered' if x == 1 else 'Lost')

In [46]:
fig = px.scatter(df.sort_values(by='customer_id'),
           x='customer_id',
           y= 'contactability_score',
          color='Strategy Result',
          color_discrete_map = {'Lost':'red', 'Recovered':'green'})

fig.update_yaxes(title="Contactability Score")
fig.update_xaxes(title="Customer Id")
